In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch  
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


# Neural Net Class and Training Functions
Define Class and functions

In [ ]:
def data_loaders():
   # Import data
    dir_X = '/Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/ev_adoption_ml/Data/df_X_county.csv'
    dir_y = '/Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/ev_adoption_ml/Data/df_y_county.csv'
    
    X = pd.read_csv(dir_X)
    X['constant'] = 1
    y = pd.read_csv(dir_y)

    # check if any nan values
    nan_row_X = X[X.isna().any(axis=1)]
    #print(nan_row_X)
    nan_row_y = y[y.isna().any(axis=1)]
    #print(nan_row_y)

    X = X.to_numpy()
    y = y.to_numpy()

    # split train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # standardize X
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

def train_model(nn_model, data_loaded, opt, batch_size=32):
    
    '''
    Trains neural network model on X_train, y_train data.
    
    Returns
    ----------
    nn_model: torch.nn.Module
        trained neural network model
    '''
    # convert to tensors (for Pytorch)
    X_train, X_eval, y_train, y_eval = data_loaded
    X_train_tensor = torch.tensor(X_train)
    y_train_tensor = torch.tensor(y_train)
    X_test_tensor = torch.tensor(X_eval)
    y_test_tensor = torch.tensor(y_eval)
    
    nn_model.train()  # put model in train mode
    # initialize mse loss function
    mse_loss = torch.nn.MSELoss()
    # train with (mini-batch) SGD; initialize optimizer
    #opt = torch.optim.SGD(nn_model.parameters(), lr=1e-4)
    n_samples, n_features = X_train_tensor.shape
    # loop through data in batches

    for batch_start in range(0, n_samples, batch_size):
    # reset gradients to zero
        opt.zero_grad()
        # form batch
        X_batch = X_train_tensor[batch_start:batch_start+batch_size]
        y_batch = y_train_tensor[batch_start:batch_start+batch_size]
        X_batch_test = X_test_tensor[batch_start:batch_start+batch_size]
        y_batch_test = y_test_tensor[batch_start:batch_start+batch_size]
        # pass batch through neural net to get prediction
        y_pred = nn_model(X_batch.float())
        y_pred = y_pred.unsqueeze(1)
        y_pred_test = nn_model(X_batch_test.float())
        y_pred_test = y_pred_test.unsqueeze(1)
        # compute MSE loss
        loss = mse_loss(y_pred, y_batch[:, None].float())
        loss_test = mse_loss(y_pred_test, y_batch_test[:, None].float())
        # back-propagate loss
        loss.backward()
        # update model parameters based on backpropogated gradients
        opt.step()
        
        #print(f"Mean Train MSE: {epoch_loss}")
        
    return epoch_loss, epoch_loss_test

def evaluate_model(nn_model, X_eval, y_eval, batch_size=32):
    '''
    Evaluates trained neural network model on X_eval, y_eval data.

    Parameters
    ----------
    nn_model: torch.nn.Module
        trained neural network model
    X_eval: np.array
        matrix of training data features
    y_eval: np.array
        vector of training data labels
    batch_size: int
        batch size to looping over dataset to generate predictions

    Returns
    ----------
    mse: float
        MSE of trained model on X_eval, y_eval data
    '''
    # initialize mse loss function
    mse_loss = torch.nn.MSELoss()
    # convert to tensors (for Pytorch)
    X_eval_tensor = torch.tensor(X_eval)
    y_eval_tensor = torch.tensor(y_eval)
    n_samples = X_eval_tensor.shape[0]
    nn_model.eval() # put in eval mode
    # loop over data and generate predictions
    preds = []
    for batch_start in range(0, n_samples, batch_size):
        # form batch
        X_batch = X_eval_tensor[batch_start:batch_start+batch_size]
        y_batch = y_eval_tensor[batch_start:batch_start+batch_size]
        with torch.no_grad():  # no need to compute gradients during evaluation
            # pass batch through neural net to get prediction
            y_pred = nn_model(X_batch.float())
            y_pred = y_pred.unsqueeze(1)
            preds.append(y_pred)
    # compute MSE across all samples
    all_preds = torch.cat(preds)
    loss = mse_loss(all_preds, y_eval_tensor[:, None].float()).item()
    return loss

def train_and_validate(config):

    '''Parameters
    ----------
    X_train: np.array
        matrix of training data features
    y_train: np.array
        vector of training data labels
    max_iter: int
        maximum number of iterations to train for
    batch_size: int
        batch size to use when training w/ SGD
    '''
    # intialize neural network
    data_loaded = data_loaders()
    X_train, X_eval, y_train, y_eval = data_loaded
    print(y_eval)
    n_samples, n_features = X_train.shape
    nn_model = NN_configureable(n_features, config["n_hidden_dim"], config["n_layers"])

    opt = torch.optim.SGD(nn_model.parameters(), lr=config["lr"],  momentum=0.9)
    batch_size = config["batch_size"]
    max_iter = config["train_iterations"]

    # Start the training.
    for it in range(max_iter):
        # save losses across all batches
        train_epoch_loss, test_epoch_loss = train_model(nn_model, data_loaded, opt, batch_size)
        valid_epoch_loss = evaluate_model(nn_model, X_eval, y_eval, batch_size)

    with tune.checkpoint_dir(it) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, 'checkpoint')
            torch.save((nn_model.state_dict(), opt.state_dict()), path)
    tune.report(
        loss=valid_epoch_loss)

class NN(nn.Module):
    '''
    Class for fully connected neural net.
    '''
    def __init__(self, input_dim, hidden_dim):
        '''
        Parameters
        ----------
        input_dim: int
            input dimension (i.e., # of features in each example passed to the network)
        hidden_dim: int
            number of nodes in hidden layer
        '''
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.layers = nn.Sequential(
            # Network has a single hidden layer
            # Apply ReLU activation in between the hidden layer and output node
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.layers(x)
        return x


class NN_configureable(nn.Module):
    '''
    Class for fully connected neural net.
    '''
    def __init__(self, input_dim, hidden_dim, hidden_layers):
        '''
        Parameters
        ----------
        input_dim: int
            input dimension (i.e., # of features in each example passed to the network)
        hidden_dim: int
            number of nodes in hidden layer
        '''
        super().__init__()
        self.input_dim = input_dim
        print(input_dim)
        self.hidden_layers = hidden_layers
        #self.layers = nn.ModuleDict()
        self.layers = nn.ModuleDict()
        
        # Define input layer
        self.layers["input"] = nn.Linear(in_features = input_dim, out_features = hidden_dim)
        # Define hidden layers
        for i in range(self.hidden_layers):
            self.layers[f"hidden_{i}"] = nn.Linear(in_features = hidden_dim, out_features = hidden_dim)
        # Define output layer
        self.layers["output"] = nn.Linear(in_features = hidden_dim, out_features = 1)

    def forward(self, x):
        x = self.layers["input"](x)
        for i in range(self.hidden_layers):
            x = F.relu(self.layers[f"hidden_{i}"](x))

        return self.layers["output"](x)        
        

# Search Function for Ray Tune
Hyperparameter search



# Import Data
Import combined data

In [18]:
def main():   
    
    # Search Function for Ray Tune - Hyperparameter search
     
    #X = pd.read_csv('./Data/df_X_county.csv')
    #print(X.head)
     
    # Define the parameter search configuration.
    config = {
        "n_layers": 
            #tune.sample_from(lambda _: 2 ** np.random.randint(1, 5)),
            tune.grid_search([1, 2, 3]),
        "n_hidden_dim": 
            #tune.sample_from(lambda _: 2 ** np.random.randint(4, 8)),
            tune.grid_search([1, 2]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([32]),
        "train_iterations": tune.choice([50, 100])
    }

    max_num_iter = 50
    grace_period = 1
    # Number of Ray Tune random search experiments to run.
    num_samples = 20
    
    # Schduler to stop bad performing trails.
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t = max_num_iter,
        grace_period = grace_period,
        reduction_factor = 2 
    )

    # Reporter to show on command line/output window
    reporter = CLIReporter(
        metric_columns=["loss", "accuracy", "training_iteration"])

    #nn_model_place = NN_configureable(input_dim, config["n_hidden_dim"], config["n_layers"])

    # Start Ray Tune search
    result = tune.run(
        train_and_validate,
        resources_per_trial = {"cpu": 2, "gpu": 0},
        config = config,
        num_samples = num_samples,
        scheduler = scheduler,
        local_dir = '../outputs/raytune_result',
        keep_checkpoints_num = 1,
        checkpoint_score_attr = 'min-validation_loss',
        progress_reporter = reporter
    )

    # Extract the best trial run from the search.
    best_trial = result.get_best_trial(
        'loss', 'min', 'last'
    )
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    
    #n_layers = np.arange(1, 5) # iterate through hidden layer count
    #n_hidden_dim = np.arange(8, 65, 8)  # iterate through hidden layer node count
    #mse_dict = {}
    
    #for i in range(len(n_layers)):
    #    for j in range(len(n_hidden_dim)):
    #        tuple_place = (i, j)
    #        nn_model_place = NN_configureable(input_dim, hidden_dim = j, hidden_layers = i)
    #        nn_model_result = train_model(nn_model_place, X_train, y_train, X_test, y_test, 32)
    #        train_mse =  evaluate_model(nn_model_result[0], X_train, y_train)
    #        test_mse = evaluate_model(nn_model_result[0], X_test, y_test)
    #        #print(tuple_place)
    #        train_test_list = [train_mse, test_mse]
    #        mse_dict[tuple_place] = train_test_list
            
        
        
    #nn_model_place = NN_configureable(input_dim, 8, 2)
    #nn_model_place = NN(input_dim, 8)
    #nn_model_result = train_model(nn_model_place, X_train, y_train, X_test, y_test, 32)
    #train_mse = evaluate_model(nn_model_result[0], X_train, y_train)
    #test_mse = evaluate_model(nn_model_result[0], X_test, y_test)
        
    #for key in mse_dict:
    #    print(f"Train MSE for model: hidden_layers = {key[0]}, hidden_dim = {key[1]} is: {mse_dict[key][0]}")
    #    print(f"Test MSE for model: hidden_layers = {key[0]}, hidden_dim = {key[1]} is: {mse_dict[key][1]}")

    # plot the model's test errors
    #plt.plot(range(len(nn_model_result[1])), nn_model_result[1])
    # axis labels

    '''
    plt.xlabel('Iteration Step')
    plt.ylabel('Test Error')
    plt.title("Model 1 - Hidden Layer - ReLU")
    plt.legend()
    plt.figure(figsize=(20,12))
    plt.show()
'''

In [19]:
if __name__ == '__main__':
    main()

2023-05-10 16:47:53,789	WARNING callback.py:142 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Current time: 2023-05-10 16:47:53 (running for 00:00:00.06)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 2.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/outputs/raytune_result/train_and_validate_2023-05-10_16-47-53
Number of trials: 16/120 (15 PENDING, 1 RUNNING)
+--------------------------------+----------+-----------------+--------------+-------------+----------------+------------+--------------------+
| Trial name                     | status   | loc             |   batch_size |          lr |   n_hidden_dim |   n_layers |   train_iterations |
|--------------------------------+----------+-----------------+--------------+-------------+----------------+------------+--------------------|
| train_and_validate_ef910_00000 | RUNNING  | 127.0.0.1:76483 |           32 | 0.000376769 |              

Trial name,date,done,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_and_validate_ef910_00000,2023-05-10_16-47-59,True,"0_batch_size=32,lr=0.0004,n_hidden_dim=1,n_layers=1,train_iterations=50",dhcp-10-29-108-17.dyn.MIT.EDU,1,1.1161e+06,127.0.0.1,76483,True,4.16121,4.16121,4.16121,1683751679,1,ef910_00000
train_and_validate_ef910_00001,2023-05-10_16-48-01,True,"1_batch_size=32,lr=0.0006,n_hidden_dim=2,n_layers=1,train_iterations=50",dhcp-10-29-108-17.dyn.MIT.EDU,1,nan,127.0.0.1,76484,True,4.21764,4.21764,4.21764,1683751681,1,ef910_00001
train_and_validate_ef910_00002,2023-05-10_16-48-02,True,,dhcp-10-29-108-17.dyn.MIT.EDU,1,1.11739e+06,127.0.0.1,76485,True,5.1529,5.1529,5.1529,1683751682,1,ef910_00002
train_and_validate_ef910_00003,2023-05-10_16-48-02,True,"3_batch_size=32,lr=0.0004,n_hidden_dim=2,n_layers=2,train_iterations=50",dhcp-10-29-108-17.dyn.MIT.EDU,1,1.11607e+06,127.0.0.1,76488,True,5.22721,5.22721,5.22721,1683751682,1,ef910_00003
train_and_validate_ef910_00004,2023-05-10_16-48-05,True,"4_batch_size=32,lr=0.0004,n_hidden_dim=1,n_layers=3,train_iterations=50",dhcp-10-29-108-17.dyn.MIT.EDU,1,1.11609e+06,127.0.0.1,76483,True,5.77656,5.77656,5.77656,1683751685,1,ef910_00004
train_and_validate_ef910_00005,2023-05-10_16-48-13,True,,dhcp-10-29-108-17.dyn.MIT.EDU,1,1.11784e+06,127.0.0.1,76484,True,11.6102,11.6102,11.6102,1683751693,1,ef910_00005
train_and_validate_ef910_00006,2023-05-10_16-48-06,True,"6_batch_size=32,lr=0.0160,n_hidden_dim=1,n_layers=1,train_iterations=50",dhcp-10-29-108-17.dyn.MIT.EDU,1,nan,127.0.0.1,76485,True,4.20862,4.20862,4.20862,1683751686,1,ef910_00006
train_and_validate_ef910_00007,2023-05-10_16-48-10,True,"7_batch_size=32,lr=0.0010,n_hidden_dim=2,n_layers=1,train_iterations=100",dhcp-10-29-108-17.dyn.MIT.EDU,1,nan,127.0.0.1,76488,True,8.38895,8.38895,8.38895,1683751690,1,ef910_00007
train_and_validate_ef910_00008,2023-05-10_16-48-10,True,,dhcp-10-29-108-17.dyn.MIT.EDU,1,1.11713e+06,127.0.0.1,76483,True,4.82228,4.82228,4.82228,1683751690,1,ef910_00008
train_and_validate_ef910_00009,2023-05-10_16-48-16,True,"9_batch_size=32,lr=0.0002,n_hidden_dim=2,n_layers=2,train_iterations=100",dhcp-10-29-108-17.dyn.MIT.EDU,1,1.11613e+06,127.0.0.1,76485,True,10.3066,10.3066,10.3066,1683751696,1,ef910_00009


2023-05-10 16:47:59,750	ERROR checkpoint_manager.py:361 -- Result dict has no key: validation_loss. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['loss', 'time_this_iter_s', 'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 'iterations_since_restore', 'experiment_tag', 'config/n_layers', 'config/n_hidden_dim', 'config/lr', 'config/batch_size', 'config/train_iterations']


== Status ==
Current time: 2023-05-10 16:47:59 (running for 00:00:05.95)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -1116098.125
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/outputs/raytune_result/train_and_validate_2023-05-10_16-47-53
Number of trials: 20/120 (16 PENDING, 4 RUNNING)
+--------------------------------+----------+-----------------+--------------+-------------+----------------+------------+--------------------+------------+----------------------+
| Trial name                     | status   | loc             |   batch_size |          lr |   n_hidden_dim |   n_layers |   train_iterations |       loss |   training_iteration |
|--------------------------------+----------+-----------------+--------------+-------------+----------------+------------+--------------------+------------+--------------

...

2023-05-10 16:51:51,337	ERROR checkpoint_manager.py:361 -- Result dict has no key: validation_loss. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['loss', 'time_this_iter_s', 'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 'iterations_since_restore', 'experiment_tag', 'config/n_layers', 'config/n_hidden_dim', 'config/lr', 'config/batch_size', 'config/train_iterations']


== Status ==
Current time: 2023-05-10 16:51:51 (running for 00:03:57.54)
Using AsyncHyperBand: num_stopped=33
Bracket: Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -1116271.125
Logical resource usage: 2.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/outputs/raytune_result/train_and_validate_2023-05-10_16-47-53
Number of trials: 120/120 (1 RUNNING, 119 TERMINATED)
+--------------------------------+------------+-----------------+--------------+-------------+----------------+------------+--------------------+---------------+----------------------+
| Trial name                     | status     | loc             |   batch_size |          lr |   n_hidden_dim |   n_layers |   train_iterations |          loss |   training_iteration |
|--------------------------------+------------+-----------------+--------------+-------------+----------------+------------+--------------------+---------

2023-05-10 16:51:51,378	INFO tune.py:945 -- Total run time: 237.60 seconds (237.57 seconds for the tuning loop).


== Status ==
Current time: 2023-05-10 16:51:51 (running for 00:03:57.58)
Using AsyncHyperBand: num_stopped=33
Bracket: Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -1116271.125
Logical resource usage: 0/8 CPUs, 0/0 GPUs
Result logdir: /Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/outputs/raytune_result/train_and_validate_2023-05-10_16-47-53
Number of trials: 120/120 (120 TERMINATED)
+--------------------------------+------------+-----------------+--------------+-------------+----------------+------------+--------------------+---------------+----------------------+
| Trial name                     | status     | loc             |   batch_size |          lr |   n_hidden_dim |   n_layers |   train_iterations |          loss |   training_iteration |
|--------------------------------+------------+-----------------+--------------+-------------+----------------+------------+--------------------+---------------+------

KeyError: 'accuracy'